CLIP 모델 임포트 및 버전 설정, GPU에 로드

In [1]:
import torch
import clip

model, preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")


COCO 데이터셋 로드

In [2]:
from torchvision.datasets import CocoCaptions
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor()])

# COCO 데이터셋 경로 설정
train_dataset = CocoCaptions(root=r'E:\CLIP_COCO\train2017',
                             annFile=r'E:\CLIP_COCO\annotations\captions_train2017.json',
                             transform=transform)

val_dataset = CocoCaptions(root=r'E:\CLIP_COCO\val2017',
                           annFile=r'E:\CLIP_COCO\annotations\captions_val2017.json',
                           transform=transform)



loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


로드된 데이터셋 확인

In [3]:
img, caption = train_dataset[0]
print("Image size:", img.size())
print("Caption:", caption)

Image size: torch.Size([3, 480, 640])
Caption: ['Closeup of bins of food that include broccoli and bread.', 'A meal is presented in brightly colored plastic trays.', 'there are containers filled with different kinds of foods', 'Colorful dishes holding meat, vegetables, fruit, and bread.', 'A bunch of trays that have different food.']


In [4]:
import clip
import torch
from PIL import Image
import numpy as np

# CLIP 모델 및 전처리 도구 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 컨텍스트 길이 설정 (CLIP의 기본값은 77)
context_length = 77

# Custom collate_fn
def custom_collate_fn(batch):
    images, captions = zip(*batch)
    processed_images = []
    processed_captions = []

    for image in images:
        # 이미지 데이터 체크 및 변환
        if isinstance(image, np.ndarray):
            # NumPy 배열을 PIL 이미지로 변환
            image = Image.fromarray(image.astype('uint8'), 'RGB')
        elif isinstance(image, torch.Tensor):
            # 텐서를 PIL 이미지로 변환
            image = Image.fromarray(image.numpy().astype('uint8'), 'RGB')
        else:
            raise TypeError("Unsupported image type")

        # 이미지 전처리
        processed_image = preprocess(image).unsqueeze(0)
        processed_images.append(processed_image)

    # 모든 이미지가 유효한지 확인
    if len(processed_images) == 0:
        raise RuntimeError("No valid images found in batch.")

    # 캡션 처리 및 길이 조정
    for caption in captions:
        if not isinstance(caption, str):
            caption = str(caption)  # 비문자열 캡션을 문자열로 변환
        # 캡션을 토큰화하고, 컨텍스트 길이 초과 시 자르기
        tokens = clip.tokenize([caption], truncate=True)[0][:context_length].tolist()
        processed_captions.append(tokens)

    images = torch.cat(processed_images, dim=0)  # 이미지 텐서 결합
    texts = torch.tensor(processed_captions).to(device)  # 캡션 텐서로 변환 및 GPU로 전송

    return images, texts



In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import clip

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

# 모델 및 전처리 도구 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 최적화 설정
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# 학습 루프
num_epochs = 1
for epoch in range(num_epochs):
    for images, texts in train_loader:
        optimizer.zero_grad()  # 기울기 초기화

        # 모델을 통해 이미지와 텍스트 피처 추출
        image_features = model.encode_image(images)
        text_features = model.encode_text(texts)

        # 유사도 계산
        logits = (image_features @ text_features.T) / 0.07  # 온도 파라미터
        labels = torch.arange(images.size(0), dtype=torch.long, device=device)

        # 대조적 손실 계산
        loss = criterion(logits, labels)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# 모델 저장
torch.save(model.state_dict(), 'trained_model.pkl')
print("모델 학습이 완료되었으며 trained_model.pkl 파일로 저장되었습니다.")
